In [ ]:
#IMAGE FILTERING AND EDGE DETECTION
import numpy as np
import cv2
# Filter Modes
PREVIEW = 0    # No filter, just preview
BLUR = 1       # Blur filter
FEATURES = 2   # Corner feature detection
CANNY = 3      # Canny Edge Detector
# PREVIEW: This mode is used to display the video feed without applying any filters.
# BLUR: In this mode, a blur filter will be applied to the video feed.
# FEATURES: This mode applies corner feature detection, which identifies key points (corners) in the image.
# CANNY: This is the mode for Canny Edge Detection, which highlights edges in the image.

# Parameters for corner feature detection
feature_params = dict(maxCorners=500,
                      qualityLevel=0.2,
                      minDistance=15,
                      blockSize=9)
# maxCorners=500: This limits the number of corners/features to be detected. The algorithm will detect at most 500 corners in the image.
# qualityLevel=0.2: This determines the minimum accepted quality of corners. A higher value means fewer corners will be detected, but they will be more distinct. A lower value results in detecting more corners, but some may be less prominent.
# minDistance=15: This ensures that detected corners are at least 15 pixels apart, preventing the algorithm from detecting multiple closely-packed corners.
# blockSize=9: This parameter defines the size of the block used for corner detection. A larger block size means the algorithm will analyze a larger neighborhood around each pixel when detecting corners.

# Set default image filter to PREVIEW mode
image_filter = PREVIEW

# Open a camera or video file
source = cv2.VideoCapture(0)  # 0 for the default camera

if not source.isOpened():
    print("Error: Cannot open the camera.")
    exit()


win_name = "Camera Filters"
cv2.namedWindow(win_name, cv2.WINDOW_NORMAL)

# Main loop to capture frames and apply filters
while True:
    ret, frame = source.read()  # Capture frame from the camera
    if not ret:
        print("Error: Unable to read frame.")
        break

    # Apply the selected filter based on user input
    if image_filter == PREVIEW:
        result = frame  # No filter applied, just preview
    elif image_filter == CANNY:
        # Convert the frame to grayscale first
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        # Apply Canny edge detection
        result = cv2.Canny(gray_frame, 80, 150)
        # The function takes two threshold values: 80 (lower) and 150 (upper). Edges with gradients higher than the upper threshold are considered "strong edges," while those between the thresholds are "weak edges." This helps eliminate noise and enhance strong edges.
    elif image_filter == BLUR:
        # Apply a Gaussian blur
        result = cv2.blur(frame, (13, 13))
        # cv2.blur() applies a Gaussian blur, which smooths the image, reducing noise. The (13, 13) tuple specifies the kernel size, meaning a 13x13 pixel neighborhood is used for the blur calculation.
    elif image_filter == FEATURES:
        # Convert frame to grayscale
        frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        # Detect corners/features in the frame
        corners = cv2.goodFeaturesToTrack(frame_gray, mask=None, **feature_params)
        # The ** operator in Python is used for unpacking a dictionary into keyword arguments. 
        # When you use **feature_params inside the function call, it "unpacks" the dictionary so that each key-value pair is passed as a separate named argument to the cv2.goodFeaturesToTrack() function.
        #eg.
        #  corners = cv2.goodFeaturesToTrack(frame_gray, maxCorners=500, qualityLevel=0.2, minDistance=15, blockSize=9)

        result = frame.copy()
        # If corners are detected, mark them on the frame
        if corners is not None:
            for x, y in np.float32(corners).reshape(-1, 2):
                cv2.circle(result, (int(x), int(y)), 10, (0, 255, 0), 1)

    # Show the resulting frame in the window
    cv2.imshow(win_name, result)

    # Wait for user input and process the filter selection
    key = cv2.waitKey(1)  # Capture keyboard input every 1 millisecond
    if key == ord('q') or key == ord('Q') or key == 27:  # Exit on 'q' or 'Esc'
        break
    elif key == ord('c') or key == ord('C'):  # Switch to Canny Edge Detection
        image_filter = CANNY
    elif key == ord('b') or key == ord('B'):  # Switch to Blur filter
        image_filter = BLUR
    elif key == ord('f') or key == ord('F'):  # Switch to Corner feature detection
        image_filter = FEATURES
    elif key == ord('p') or key == ord('P'):  # Switch back to Preview mode
        image_filter = PREVIEW

# Clean up
source.release()
cv2.destroyAllWindows()
